# Stacking

In [1]:
import numpy as np # linear algebra
import pandas as pd 
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import linear_model
import tensorflow as tf

# Write prediction to matrix

In [2]:
def stacking_feature(file):
    from sklearn.feature_extraction import DictVectorizer
    lines=open(file).readlines()
    result=[]
    file_id=[]
    for e,line in enumerate(lines):
        if e>0:
            row=line.strip().split(",")
            string_list=row[1].split(" ")
            float_list=[float(string_list[x]) if x%2==1 else string_list[x] for x in range(0,len(string_list))]
            newlist=[float_list[x:x+2] for x in range(0,len(float_list),2)]
            score_dict={}
            for x in range(0,len(newlist)):
                score_dict[newlist[x][0]]=(newlist[x][1])
            result.append(score_dict)
            file_id.append(row[0])
    feature=DictVectorizer(sparse=False).fit_transform(result)
    return file_id,feature

In [3]:
def _int64list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value)) 


class NumpyToTFHelper(object):
    def __init__(self,filenames):
        self.filenames = filenames
        self.current_file = -1
        self.current_index = 0 
        self.current_array = None
    def get_next(self):
        if self.current_file<len(self.filenames)-1:
            self.current_file += 1
            self.current_array = self.filenames[self.current_file]
        else:
            return None
        self.current_index+=1
        cur = self.filenames[self.current_index -1] 
        return np.matrix(cur).A1


## labels

In [4]:
val_y = pd.read_csv("/Users/shujiaohuang_Mac/Downloads/validate_labels.csv",header=None).as_matrix()

## X_train

In [8]:
moe_2feature_eval_stacking_new_feature_withid=stacking_feature('.../moe_2feature_eval_predictions.csv')

In [24]:
moe_2feature_eval_stacking_new_feature=moe_2feature_eval_stacking_new_feature_withid[1]
moe_2feature_eval_stacking_new_feature

array([[ 0.682859,  0.044098,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.978585,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.999238,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       ..., 
       [ 0.999989,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.01214 ,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.654479,  0.982899,  0.      , ...,  0.      ,  0.      ,  0.      ]])

In [25]:
moe_2feature_eval_stacking_new_feature.shape

(1401828, 4716)

In [31]:
num=100000
for iter in range(1,16):
    if iter<moe_2feature_eval_stacking_new_feature.shape[0]/num:
        X_top=moe_2feature_eval_stacking_new_feature[int(iter-1)*num:int(iter)*num]
        y_top=val_y[int(iter-1)*num:int(iter)*num]
    else:
        X_top=moe_2feature_eval_stacking_new_feature[int(iter-1)*num::]
        y_top=val_y[int(iter-1)*num::]
    filename='moe_val_10w_'+str(iter)+'.tfrecord'
    writer = tf.python_io.TFRecordWriter(filename)
    N_X = NumpyToTFHelper(X_top) 
    n_X = N_X.get_next()
    N_y = NumpyToTFHelper(y_top)
    n_y = N_y.get_next()
    while n_X is not None:
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    'video_id':_bytes_feature(bytes(n_y[0],'utf-8')),
                    'labels':_int64list_feature([int(i) for i in n_y[1].split(' ')]),
                    'New_feature':_floats_feature([float(i) for i in n_X])
                }
            )
        ) 
        writer.write(example.SerializeToString())
        n_X=N_X.get_next()
        n_y = N_y.get_next()
    writer.close()

In [4]:
logistic_2feature_eval_stacking_new_feature_withid=stacking_feature('/Users/shujiaohuang_Mac/Downloads/logistic_val_predictions.csv')

In [24]:
logistic_2feature_eval_stacking_new_feature=logistic_2feature_eval_stacking_new_feature_withid[1]

In [135]:
logistic_2feature_eval_stacking_new_feature

array([[ 0.907931,  0.02949 ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.976476,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.999979,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       ..., 
       [ 0.999975,  0.016853,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.048551,  0.004482,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.465478,  0.954355,  0.      , ...,  0.      ,  0.      ,  0.      ]])

In [144]:
# Both moe and logisitc val tfrecord
for iteration in range(1,16):
    print(iteration)
    file_iter='10w_'+str(iteration)
    num=100000
    #file_iter='10w_'+str(iteration)
    moe_val='/Users/shujiaohuang_Mac/Downloads/moe_val_'+str(file_iter)+'.tfrecord'
    filename='both_moe_logistic_val_'+str(file_iter)+'.tfrecord'
    writer = tf.python_io.TFRecordWriter(filename)
    if iter<logistic_2feature_eval_stacking_new_feature.shape[0]/num:
        logistic_feature_temp=logistic_2feature_eval_stacking_new_feature[int(iteration-1)*num:int(iteration)*num]
    else:
        logistic_feature_temp=logistic_2feature_eval_stacking_new_feature[int(iteration-1)*num::]
    N_logistic=NumpyToTFHelper(logistic_feature_temp) 
    n_logistic = N_logistic.get_next()
    for example in tf.python_io.tf_record_iterator(moe_val):
        moe_example=tf.train.Example.FromString(example)
        example_both_moe_logistic=tf.train.Example(
            features=tf.train.Features(
                feature={
                    'video_id':moe_example.features.feature['video_id'],
                    'labels':moe_example.features.feature['labels'],
                    'moe_newfeature':moe_example.features.feature['New_feature'],
                    'logistic_newfeature':_floats_feature([float(i) for i in n_logistic])
                }
            )
        )
        writer.write(example_both_moe_logistic.SerializeToString())
        n_logistic=N_logistic.get_next()
    writer.close()

3
4
5
6
7
8
9
10
11
12
13
14
15


## X_test

In [ ]:
moe_2feature_test_blending_new_feature_withid=stacking_feature('.../moe_test_predictions.csv')
test_id=moe_2feature_test_blending_new_feature_withid[0]
feature=moe_2feature_test_blending_new_feature_withid[1]

In [52]:
# moe
num=50000
for iter in range(1,16):
    if iter<feature.shape[0]/num:
        test_id_temp=np.array([test_id[int(iter-1)*num:int(iter)*num]])
        feature_temp=feature[int(iter-1)*num:int(iter)*num]
    else:
        test_id_temp=np.array([test_id[int(iter-1)*num::]])
        feature_temp=feature[int(iter-1)*num::]
    filename='moe_test_5w_'+str(iter)+'.tfrecord'
    writer = tf.python_io.TFRecordWriter(filename)
    N_feature = NumpyToTFHelper(feature_temp) 
    n_feature = N_feature.get_next()
    N_id=NumpyToTFHelper(test_id_temp.T) 
    n_id = N_id.get_next()
    while n_id is not None:
        raw=n.tostring()
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    'video_id':_bytes_feature(bytes(n_id[0],'utf-8')),
                    'labels':_int64list_feature([]),
                    'New_feature':_floats_feature([float(i) for i in n_feature])
                }
            )
        ) 
        writer.write(example.SerializeToString())
        n_feature=N_feature.get_next()
        n_id=N_id.get_next()
    writer.close()

In [4]:
logistic_2feature_test_blending_new_feature_withid=stacking_feature('/Users/shujiaohuang_Mac/Downloads/logistic_test_predictions.csv')
logistic_2feature_test_blending_new_feature=logistic_2feature_test_blending_new_feature_withid[1]

In [32]:
# Both moe and logisitc test tfrecord
for iteration in range(1,16):
    print(iteration)
    num=50000
    moe_test='/Users/shujiaohuang_Mac/Downloads/moe_test_all/moe_test_5w_'+str(iteration)+'.tfrecord'
    filename='both_moe_logistic_test_5w_'+str(iteration)+'.tfrecord'
    writer = tf.python_io.TFRecordWriter(filename)
    if iteration<logistic_2feature_test_blending_new_feature.shape[0]/num:
        logistic_feature_temp=logistic_2feature_test_blending_new_feature[int(iteration-1)*num:int(iteration)*num]
    else:
        logistic_feature_temp=logistic_2feature_test_blending_new_feature[int(iteration-1)*num::]
    N_logistic=NumpyToTFHelper(logistic_feature_temp) 
    n_logistic = N_logistic.get_next()
    for example in tf.python_io.tf_record_iterator(moe_test):
        moe_example=tf.train.Example.FromString(example)
        example_both_moe_logistic=tf.train.Example(
            features=tf.train.Features(
                feature={
                    'video_id':moe_example.features.feature['video_id'],
                    'labels':moe_example.features.feature['labels'],
                    'moe_newfeature':moe_example.features.feature['New_feature'],
                    'logistic_newfeature':_floats_feature([float(i) for i in n_logistic])
                }
            )
        )
        writer.write(example_both_moe_logistic.SerializeToString())
        n_logistic=N_logistic.get_next()
    writer.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
